In [50]:
import numpy as np
import pandas as pd
import duckdb

In [51]:
from datasets.diabetes import Diabetes

ds = Diabetes("../data/diabetes")

In [52]:
train_ds = ds.train_data()

In [53]:
train_ds[0][0].shape

In [54]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import AdamW

In [55]:
model = nn.Sequential(
    nn.Linear(21, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
)

In [56]:
loss_fn = nn.BCELoss()
optimizer = AdamW(model.parameters())

In [57]:
train_dl = DataLoader(train_ds, batch_size=64)

In [59]:
model.train()
for epoch in range(100):
    epoch_loss = 0
    for X, y in train_dl:
        model.zero_grad()
        
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.cpu().item()
        
    epoch_loss /= len(train_dl)
    print(f"Epoch {epoch}: loss={epoch_loss:.4f}")

In [ ]:
model.eval()
test_loss = 0
test_corr = 0

for X, y in train_dl:
    with torch.no_grad():        
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        
        test_loss += loss.cpu().item()
        test_corr += ((y_pred > 0.5) == y).sum()
            

test_loss /= len(train_dl)
test_acc = test_corr / len(train_dl.dataset)

In [ ]:
test_loss

In [ ]:
test_acc